#### Exploring Which Dataset to Use

In [1]:
import pandas as pd
import os
import numpy as np
from difflib import get_close_matches

os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Coding Projects/woba modeling/data/')

#### Load Data

In [2]:
m1_woba = pd.read_csv('sim_results/m1_wobacon_23_25.csv')
m2_woba = pd.read_csv('sim_results/m2_wobacon_23_25.csv')
ev_dir = pd.read_csv('sim_results/ev_wobacon_23_25.csv')
full = pd.read_csv('transforms/wobacon.csv')
woba = pd.read_csv('pitch/woba/savant.csv')

#### Correlations

In [3]:
merged = woba.merge(full[['name', 'year', 'count', 'monte']], on=['name', 'year'], how='right')
merged = merged.merge(ev_dir[['name', 'year', 'monte']], on =  ['name', 'year'], how='right', suffixes=["", "_ev"])
# merged = merged.merge(ev_dir[['name', 'year', 'monte']], on = ['name', 'year'], how='right', suffixes=('', '_ev'))
# merged = merged.merge(m2_woba[['name', 'year', 'count', 'monte']], on=['name', 'year'], how='right', suffixes=('', '_m2'))
# merged = merged.rename(columns={'count': 'count_m1', 'monte': 'm1_monte', 'count_m2': 'count_m2', 'monte_m2': 'm2_monte'})

In [4]:
missing_names = merged[merged.isnull().any(axis=1)]['name'].unique()
existing_names = merged[~merged.isnull().any(axis=1)]['name'].unique()

name_map = {}
for name in missing_names:
    matches = get_close_matches(name, existing_names, n=1, cutoff=0.8)
    if matches:
        name_map[name] = matches[0]



In [5]:
for missing_name, matched_name in name_map.items():
    idx_missing = merged[(merged['name'] == missing_name) & (merged.isnull().any(axis=1))].index
    for idx in idx_missing:
        year = merged.loc[idx, 'year']
        match_row = merged[(merged['name'] == matched_name) & (merged['year'] == year) & (~merged.isnull().any(axis=1))]
        if not match_row.empty:
            for col in merged.columns:
                if pd.isnull(merged.at[idx, col]):
                    merged.at[idx, col] = match_row.iloc[0][col]

In [6]:
merged = merged[['name', 'year', 'monte','wobacon', 'xwobacon', 'pa', 'count']]

In [7]:
merged = merged[merged['count'] > 60]

#### Correlation

stability

In [8]:
stats = ['monte', 'wobacon', 'xwobacon']
corr_next = merged.copy()

for stat in stats:
    corr_next[f'{stat}_next'] = corr_next.groupby('name')[stat].shift(-1)

corr_next = corr_next.dropna(subset=[f'{stat}_next' for stat in stats])

corrs = {}
for stat in stats:
    corrs[stat] = corr_next[[stat, f'{stat}_next']].corr().iloc[0, 1]
    corrs[stat] = corrs[stat] ** 2

print(corrs)

{'monte': 0.5949524005378153, 'wobacon': 0.1492282838393763, 'xwobacon': 0.42430711474986027}


dataset to next year wobacon

In [9]:
prev_stat_next_woba_corr = {}
for stat in stats:
    prev_stat_next_woba_corr[stat] = corr_next[[stat, 'wobacon_next']].corr().iloc[0, 1]
    prev_stat_next_woba_corr[stat] = prev_stat_next_woba_corr[stat]

print(prev_stat_next_woba_corr)

{'monte': 0.5487283011118765, 'wobacon': 0.3863007686238487, 'xwobacon': 0.507934559672309}


2023 stat to 2024 wobacon

In [10]:
stat_2024_to_2025_woba_corr = {}
df_2024 = corr_next[corr_next['year'] == 2023]
for stat in stats:
    stat_2024_to_2025_woba_corr[stat] = df_2024[[stat, 'wobacon_next']].corr().iloc[0, 1]
    stat_2024_to_2025_woba_corr[stat] = stat_2024_to_2025_woba_corr[stat] ** 2

print(stat_2024_to_2025_woba_corr)

{'monte': 0.31056958940795243, 'wobacon': 0.13912817118964124, 'xwobacon': 0.25991914300966396}


2024 stat to 2025 wobacon

In [11]:
stat_2024_to_2025_woba_corr = {}
df_2024 = corr_next[corr_next['year'] == 2024]
for stat in stats:
    stat_2024_to_2025_woba_corr[stat] = df_2024[[stat, 'wobacon_next']].corr().iloc[0, 1]
    stat_2024_to_2025_woba_corr[stat] = stat_2024_to_2025_woba_corr[stat] ** 2

print(stat_2024_to_2025_woba_corr)

{'monte': 0.2966551270304057, 'wobacon': 0.1684289757701312, 'xwobacon': 0.2658291906779023}


xwobacon to monte

In [12]:
print(merged['monte'].corr(merged['xwobacon']))
print(merged['monte'].corr(merged['wobacon']))
print(merged['wobacon'].corr(merged['xwobacon']))


0.9056288434867004
0.7365518225141752
0.8181850330996727


#### Diffrences

In [13]:
merged['diff'] = abs(merged['monte'] - merged['xwobacon'])
print(merged['diff'].nlargest(10))

906     0.161834
145     0.118892
1695    0.104817
1331    0.095451
1357    0.095292
1642    0.092242
250     0.092216
248     0.089697
1390    0.088653
940     0.083844
Name: diff, dtype: float64


In [15]:
merged = merged[['name', 'year', 'monte', 'wobacon', 'xwobacon', 'diff']]
merged.to_excel('model.xlsx')